In [1]:
import xarray as xr

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    ## Load CM4p125's state variable output
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_rho2/"
    umo = xr.open_mfdataset(f"{zdiag_path}ts/annual/10yr/*0341-0350.umo.nc", chunks={'time':1}).sum('rho2_l').mean('time')
    vmo = xr.open_mfdataset(f"{zdiag_path}ts/annual/10yr/*0341-0350.vmo.nc", chunks={'time':1}).sum('rho2_l').mean('time')
    trans = xr.merge([umo, vmo])
    
    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc")
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    trans = trans.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                   'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
                                   'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
                                   'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
                                   'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
                                   'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
                                   'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
                                   'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])})
    
    trans.drop(['rho2_i', 'nv', 'average_DT']).to_netcdf("/work/hfd/datasets/regionate/example_diagnostics/CM4p125_barotropic_transport.nc", mode="w")